# Control Plane: Monitoring with the Azure AI Foundry SDK

----
This notebook focuses on the **Control Plane** perspective in Azure AI Foundry and shows how to do lightweight checks with the **Foundry SDK (2.0.0b2 preview)**.

You will learn how to:

- Inspect deployed agents (Fleet-style view)
- Review Assets (Connections, Agents)
- Understand where Quota / Rate limiting is managed (mostly in Portal)

## Table of contents

- [Control Plane overview](#control-plane-overview)
- [Portal vs SDK](#portal-vs-sdk)
- [Setup](#setup)
- [Case 1: Fleet overview](#case-1-fleet-overview)
- [Case 2: Assets](#case-2-assets)
- [Case 3: Quota](#case-3-quota)
- [Additional resources](#additional-resources)
- [Wrap-up](#wrap-up)

## Control Plane overview

### What is the Control Plane?

The Control Plane is the centralized operational surface in Azure AI Foundry for monitoring and governing AI resources across projects.

```
Control Plane = monitoring + management + security + governance
```

### Key areas



## Portal vs SDK

| Feature | Portal | SDK (Python) |
|-----|--------|-------------|
| Fleet overview | ✅ dashboards | ⚠️ limited listing |
| Assets view | ✅ visual | ✅ programmatic |
| Compliance dashboards | ✅ real-time | ❌ not supported |
| Quota management | ✅ manage in Portal | ❌ often read-only/limited |
| Admin / RBAC | ✅ manage access | ❌ not supported |

**Recommended usage:**
- Portal: real-time monitoring, compliance/security checks, quota management
- SDK: automation, inventory checks, CI/CD validations

# Setup

This notebook reuses the configuration file (`.foundry_config.json`) created by `0_setup/1_setup.ipynb`.

- If the file is missing, run the setup notebook first.
- Make sure you can authenticate (e.g., `az login`), so `DefaultAzureCredential` can work.

In [9]:
# Ensure the notebook kernel can find Azure CLI (`az`) on PATH
import os
import subprocess
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

load_dotenv(override=True)

possible_paths = [
    '/opt/homebrew/bin',  # macOS (Apple Silicon)
    '/usr/local/bin',     # macOS (Intel) / Linux
    '/usr/bin',           # Linux / Codespaces
    '/home/linuxbrew/.linuxbrew/bin',  # Linux Homebrew
]

az_path = None
try:
    result = subprocess.run(['which', 'az'], capture_output=True, text=True)
    if result.returncode == 0:
        az_path = os.path.dirname(result.stdout.strip())
        print(f'🔍 Azure CLI found: {result.stdout.strip()}')
except Exception:
    pass

paths_to_add: list[str] = []
if az_path and az_path not in os.environ.get('PATH', ''):
    paths_to_add.append(az_path)
else:
    for path in possible_paths:
        if os.path.exists(path) and path not in os.environ.get('PATH', ''):
            paths_to_add.append(path)

if paths_to_add:
    os.environ['PATH'] = ':'.join(paths_to_add) + ':' + os.environ.get('PATH', '')
    print(f"✅ Added to PATH: {', '.join(paths_to_add)}")
else:
    print('✅ PATH looks good already')

print(f"\nPATH (first 150 chars): {os.environ['PATH'][:150]}...")

🔍 Azure CLI found: /anaconda/envs/azureml_py38/bin//az
✅ PATH looks good already

PATH (first 150 chars): /anaconda/envs/azureml_py38/bin/:/afh/code/agent-operator-lab/.venv/bin:/home/azureuser/.vscode-server/cli/servers/Stable-994fd12f8d3a5aa16f17d42c041e...


In [10]:
# Load Foundry project settings from .foundry_config.json
import json
import os

config_file = '../0_setup/.foundry_config.json'
try:
    with open(config_file, 'r', encoding='utf-8') as f:
        config = json.load(f)
except FileNotFoundError as e:
    print(f"⚠️ Could not find '{config_file}'.")
    print('💡 Run 0_setup/1_setup.ipynb first to create it.')
    raise e

FOUNDRY_NAME = config.get('FOUNDRY_NAME')
RESOURCE_GROUP = config.get('RESOURCE_GROUP')
LOCATION = config.get('LOCATION')
TENANT_ID = config.get('TENANT_ID')
PROJECT_NAME = config.get('PROJECT_NAME', 'proj-default')
PROJECT_ENDPOINT = config.get('FOUNDRY_ENDPOINT')

os.environ['FOUNDRY_NAME'] = FOUNDRY_NAME or ''
os.environ['LOCATION'] = LOCATION or ''
os.environ['RESOURCE_GROUP'] = RESOURCE_GROUP or ''
os.environ['AZURE_SUBSCRIPTION_ID'] = config.get('AZURE_SUBSCRIPTION_ID', '')
os.environ['_ENDPOINT'] = PROJECT_ENDPOINT or ''
os.environ['PROJECT_ENDPOINT'] = PROJECT_ENDPOINT or ''

print(f"✅ Loaded settings from '{config_file}'.")
print(f"\n📌 Foundry name: {FOUNDRY_NAME}")
print(f"📌 Resource group: {RESOURCE_GROUP}")
print(f"📌 Location: {LOCATION}")
print(f"📌 Project endpoint: {PROJECT_ENDPOINT}")

# Fleet overview - list deployed agents
credential = DefaultAzureCredential()
client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)

✅ Loaded settings from '../0_setup/.foundry_config.json'.

📌 Foundry name: foundry-rq90gs
📌 Resource group: foundry-rg
📌 Location: swedencentral
📌 Project endpoint: https://foundry-rq90gs.services.ai.azure.com/api/projects/default-project


# Case 1: Fleet overview

From a fleet perspective, list **deployed agents** in the project. (The SDK does not expose every Portal dashboard field, but it is useful for automated checks.)

In [11]:
import inspect

# Create a simple agent in the current Foundry project (best-effort across SDK method names)

agent_name = f"cp-monitor-agent-{PROJECT_NAME}"
model_name = "gpt-4.1-mini"

instructions = (
    "You are a control-plane monitoring assistant for Azure AI Foundry. "
    "Help users inventory deployed agents and project assets, and summarize results clearly."
)

# Optional tools (only passed if the SDK method supports it)
tools = [{"type": "code_interpreter"}]

# Find a compatible create method on the SDK surface
create_method_name = next(
    (n for n in ("create_agent", "create", "begin_create", "create_or_update") if hasattr(client.agents, n)),
    None,
)

if not create_method_name:
    print("⚠️ Could not find an agent creation method on client.agents.")
    print("Available members:", [m for m in dir(client.agents) if "create" in m.lower()])
else:
    create_fn = getattr(client.agents, create_method_name)
    sig = inspect.signature(create_fn)

    # Build kwargs only from parameters supported by this SDK version
    call_kwargs = {}
    if "name" in sig.parameters:
        call_kwargs["name"] = agent_name
    elif "agent_name" in sig.parameters:
        call_kwargs["agent_name"] = agent_name

    if "model" in sig.parameters:
        call_kwargs["model"] = model_name
    elif "model_id" in sig.parameters:
        call_kwargs["model_id"] = model_name

    if "instructions" in sig.parameters:
        call_kwargs["instructions"] = instructions

    if "tools" in sig.parameters:
        call_kwargs["tools"] = tools

    try:
        agent = create_fn(**call_kwargs)
        print(f"✅ Created agent via client.agents.{create_method_name}()")
        print(f"   Name: {getattr(agent, 'name', agent_name)}")
        print(f"   ID:   {getattr(agent, 'id', 'N/A')}")
        print(f"   Model:{getattr(agent, 'model', getattr(agent, 'model_id', model_name))}")
    except Exception as e:
        print(f"⚠️ Agent creation failed: {e}")
        print("💡 If this fails due to model/connection requirements, set FOUNDRY_DEFAULT_MODEL to a valid deployed model name.")

⚠️ Agent creation failed: missing required argument: definition
💡 If this fails due to model/connection requirements, set FOUNDRY_DEFAULT_MODEL to a valid deployed model name.


## Case 1.1: List agents
---
Connect to the project endpoint, then call `client.agents.list()`.

In [5]:


try:
    agents = list(client.agents.list())

    print('🚀 Fleet overview - Agents')
    print('=' * 80)

    if not agents:
        print('\n⚠️ No deployed agents found.')
        print('💡 Create/deploy an agent first, then rerun this cell.')
    else:
        print(f'\n📊 {len(agents)} agent(s) found:\n')

        for i, agent in enumerate(agents, 1):
            print(f'{i}. {agent.name}')
            print(f'   ID: {agent.id}')

            model_name = getattr(agent, 'model', None) or getattr(agent, 'model_id', 'N/A')
            print(f'   Model: {model_name}')

            if hasattr(agent, 'tools') and agent.tools:
                tools_str = ', '.join([t.get('type', 'unknown') for t in agent.tools])
                print(f'   Tools: {tools_str}')

            if hasattr(agent, 'created_at'):
                print(f'   Created: {agent.created_at}')

            print('   Status: ✅ Active')
            print('-' * 80)

        print('\n✅ Fleet overview listing completed.')

except Exception as e:
    print(f'⚠️ Failed to list agents: {e}')
    print('\nTroubleshooting:')
    print('   1. Ensure you are signed in (az login)')
    print('   2. Verify PROJECT_ENDPOINT is correct')
    print('   3. Check RBAC permissions on the project')

🚀 Fleet overview - Agents

⚠️ No deployed agents found.
💡 Create/deploy an agent first, then rerun this cell.


# Case 2: Assets

List **Connections** and **Agents**, then visualize simple distributions.

## Case 2.1: List Connections and Agents
---
This is the SDK equivalent of checking Build > Assets in the Portal.

In [ ]:
# Assets - list connections and agents
print('📦 Assets')
print('=' * 80)

try:
    # 1) Connections
    print('\n1️⃣ Connections:')
    print('-' * 80)

    connections = list(client.connections.list())

    if not connections:
        print('⚠️ No connections found.')
        print('💡 Deploy/connect a model in the Portal: https://ai.azure.com')
    else:
        for conn in connections:
            print(f'\n✅ {conn.name}')
            if hasattr(conn, 'connection_type'):
                print(f'   Type: {conn.connection_type}')
            if hasattr(conn, 'endpoint_url'):
                print(f'   Endpoint: {conn.endpoint_url}')

    # 2) Agents
    print('\n\n2️⃣ Agents:')
    print('-' * 80)

    agents = list(client.agents.list())
    if agents:
        for agent in agents:
            print(f'\n✅ {agent.name}')
            print(f'   ID: {agent.id}')
            model_name = getattr(agent, 'model', None) or getattr(agent, 'model_id', 'N/A')
            print(f'   Model: {model_name}')
    else:
        print('⚠️ No agents found.')

    print('\n' + '=' * 80)
    print('✅ Assets listing completed.')

except Exception as e:
    print(f'\n⚠️ Failed to list assets: {e}')
    print('💡 Check in Portal: https://ai.azure.com > Build > Assets')

## Case 2.2: Quick visualization
---
Plot distributions for connection types and agent models.

In [ ]:
import matplotlib.pyplot as plt

try:
    connections = list(client.connections.list())
    agents = list(client.agents.list())

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Connection types distribution
    if connections:
        connection_types: dict[str, int] = {}
        for conn in connections:
            if hasattr(conn, 'connection_type'):
                conn_type = str(conn.connection_type)
                connection_types[conn_type] = connection_types.get(conn_type, 0) + 1

        if connection_types:
            ax1.bar(connection_types.keys(), connection_types.values(), color='skyblue')
            ax1.set_title('Connection Types Distribution', fontsize=14, fontweight='bold')
            ax1.set_xlabel('Connection Type')
            ax1.set_ylabel('Count')
            ax1.tick_params(axis='x', rotation=45)
        else:
            ax1.text(0.5, 0.5, 'No typed connections', ha='center', va='center', fontsize=12)
            ax1.set_title('Connection Types Distribution', fontsize=14, fontweight='bold')
    else:
        ax1.text(0.5, 0.5, 'No connections available', ha='center', va='center', fontsize=12)
        ax1.set_title('Connection Types Distribution', fontsize=14, fontweight='bold')

    # Agent models distribution
    if agents:
        agent_models: dict[str, int] = {}
        for agent in agents:
            model_name = getattr(agent, 'model', None) or getattr(agent, 'model_id', 'Unknown')
            agent_models[str(model_name)] = agent_models.get(str(model_name), 0) + 1

        ax2.bar(agent_models.keys(), agent_models.values(), color='lightcoral')
        ax2.set_title('Agent Models Distribution', fontsize=14, fontweight='bold')
        ax2.set_xlabel('Model')
        ax2.set_ylabel('Count')
        ax2.tick_params(axis='x', rotation=45)
    else:
        ax2.text(0.5, 0.5, 'No agents available', ha='center', va='center', fontsize=12)
        ax2.set_title('Agent Models Distribution', fontsize=14, fontweight='bold')

    plt.tight_layout()
    plt.show()

    print('📊 Visualization completed.')

except Exception as e:
    print(f'⚠️ Visualization failed: {e}')

# Case 3: Quota

Quota (e.g., TPM/RPM/PTU) and live usage are typically reviewed and managed in the **Azure Portal / Foundry Portal**.

**Note:** detailed quota management is usually Portal-only.
- Operate > Quota: monitor real-time usage
- Request quota increases

In [ ]:
# Quota and rate limiting pointers
print('📊 Quota and Rate Limiting')
print('=' * 80)
print('\n💡 Quota and rate limits are primarily managed in the Azure Portal.')
print('\nWhere to check:')
print('   1. Azure Portal: https://portal.azure.com')
print('   2. Select your AI Services / Foundry resource')
print("   3. Open 'Quota' or 'Usage + quotas'")
print('\nCommon fields:')
print('   - Token Per Minute (TPM)')
print('   - Requests Per Minute (RPM)')
print('   - Provisioned Throughput Units (PTU)')
print('\n' + '=' * 80)
print('✅ Use the Portal for real-time usage and quota changes.')

## Additional resources

- [Azure AI Foundry Control Plane overview](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/overview?view=foundry)
- [Monitoring across fleet](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/monitoring-across-fleet?view=foundry)
- [Register a custom agent](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/register-custom-agent?view=foundry)
- [Quickstart: create a guardrail policy](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/quickstart-create-guardrail-policy?view=foundry)
- [Manage compliance and security](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/how-to-manage-compliance-security?view=foundry)
- [Optimize cost and performance](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/how-to-optimize-cost-performance?view=foundry)

# Wrap-up

In this notebook, you did a quick Control Plane-style check using the SDK.

- Good fit for SDK: listing agents/connections, scripting inventory checks
- Portal-first tasks: quota changes, compliance/security dashboards, admin controls

Suggested next steps:
1. Review Fleet/Quota dashboards in the Portal
2. Automate periodic inventory checks in CI
3. Configure governance controls (guardrails/compliance) for production